##### Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Asynchronous Python requests

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Asynchronous_requests.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook will show you how to make asynchronous and parallel requests using the Gemini API's Python SDK and Python 3's [`asyncio`](https://docs.python.org/3/library/asyncio.html) standard library.

The examples here run in Google Colab and use the implicit event loop supplied in Colab. You can also run these commands interactively using the `asyncio` REPL (invoked with `python -m asyncio`), or you can manage the [event loop](https://docs.python.org/3/library/asyncio-eventloop.html) yourself.

In [5]:
%pip install -qU 'google-genai>=1.0.0' aiohttp

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: "'google-genai": Expected package name at the start of dependency specifier
    'google-genai
    ^


## Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](../quickstarts/Authentication.ipynb) quickstart for an example.

In [1]:
from google import genai

client = genai.Client()

Now select the model you want to use in this guide, either by selecting one in the list or writing it down. Keep in mind that some models, like the 2.5 ones are thinking models and thus take slightly more time to respond (cf. [thinking notebook](./Get_started_thinking.ipynb) for more details and in particular learn how to switch the thiking off).

In [2]:
MODEL_ID = "gemini-2.5-flash" # @param ["gemini-2.5-flash-lite","gemini-2.0-flash","gemini-2.5-flash","gemini-2.5-pro"] {"allow-input":true, isTemplate: true}

## Using local files

This simple example shows how can you use local files (presumed to load quickly) with the SDK's `async` API.

In [3]:
prompt = "Describe this image in just 3 words."

img_filenames = ["firefighter.jpg", "elephants.jpeg", "jetpack.jpg"]
img_dir = "https://storage.googleapis.com/generativeai-downloads/images/"

Start by downloading the files locally.

In [6]:
!wget -nv {img_dir}{{{','.join(img_filenames)}}}

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
ERROR: cannot verify storage.googleapis.com's certificate, issued by `/C=US/O=Google Trust Services/CN=WR2':
  Unable to locally verify the issuer's authority.
ERROR: certificate common name `*.storage.googleapis.com' doesn't match requested host name `storage.googleapis.com'.
To connect to storage.googleapis.com insecurely, use `--no-check-certificate'.
Unable to establish SSL connection.


The async code uses the `aio.models.generate_content` method to invoke the API. Most async API methods can be found in the [`aio`](https://googleapis.github.io/python-genai/genai.html#genai.client.AsyncClient) namespace.

Note that this code is not run in parallel. The async call indicates that the event loop *can* yield to other tasks, but there are no other tasks scheduled in this code. This may be sufficient, e.g. if you are running this in a web server request handler as it will allow the handler to yield to other tasks while waiting for the API response.

In [8]:
import PIL

async def describe_local_images():

  for img_filename in img_filenames:

    with open(img_filename, 'rb') as f:
      image_bytes = f.read()
    r = await client.aio.models.generate_content(
        model=MODEL_ID,
        contents=[prompt, image_bytes]
    )
    print(r.text)


await describe_local_images()

ValidationError: 12 validation errors for _GenerateContentParameters
contents.Content
  Input should be a valid dictionary or object to extract fields from [type=model_attributes_type, input_value=['Describe this image in ...0\xa2\x8a(\x03\xff\xd9'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/model_attributes_type
contents.str
  Input should be a valid string [type=string_type, input_value=['Describe this image in ...0\xa2\x8a(\x03\xff\xd9'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
contents.File
  Input should be a valid dictionary or object to extract fields from [type=model_attributes_type, input_value=['Describe this image in ...0\xa2\x8a(\x03\xff\xd9'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/model_attributes_type
contents.Part
  Input should be a valid dictionary or object to extract fields from [type=model_attributes_type, input_value=

## Downloading images asynchronously and in parallel

This example shows a more real-world case where an image is downloaded from an external source using the async HTTP library [`aiohttp`](https://pypi.org/project/aiohttp), and each image is processed in parallel.

In [8]:
import io, aiohttp, asyncio

async def download_image(session: aiohttp.ClientSession, img_url: str) -> PIL.Image:
  """Returns a PIL.Image object from the provided URL."""
  async with session.get(img_url) as img_resp:
    buffer = io.BytesIO()
    buffer.write(await img_resp.read())
    return PIL.Image.open(buffer)


async def process_image(img_future: asyncio.Future[PIL.Image]) -> str:
  """Summarise the image using the Gemini API."""
  # This code uses a future so that it defers work as late as possible. Using a
  # concrete Image object would require awaiting the download task before *queueing*
  # this content generation task - this approach chains the futures together
  # so that the download only starts when the generation is scheduled.
  r = await client.aio.models.generate_content(
      model=MODEL_ID,
      contents=[prompt, await img_future]
  )
  return r.text

In [9]:
async def download_and_describe():

  async with aiohttp.ClientSession() as sesh:
    response_futures = []
    for img_filename in img_filenames:

      # Create the image download tasks (this does not schedule them yet).
      img_future = download_image(sesh, img_dir + img_filename)

      # Kick off the Gemini API request using the pending image download tasks.
      text_future = process_image(img_future)

      # Save the reference so they can be processed as they complete.
      response_futures.append(text_future)

    print(f"Download and content generation queued for {len(response_futures)} images.")

    # Process responses as they complete (may be a different order). The tasks are started here.
    for response in asyncio.as_completed(response_futures):
      print()
      print(await response)


await download_and_describe()

Download and content generation queued for 3 images.

Wild elephant family.

Jetpack backpack concept

Cat, person, tree.


In the above example, a coroutine is created for each image that both downloads and then summarizes the image. The coroutines are executed in the final step, in the `as_completed` loop. To start them as early as possible without blocking the other work, you could wrap `download_image` in [`asyncio.ensure_future`](https://docs.python.org/3/library/asyncio-future.html#asyncio.ensure_future), but for this example the execution has been deferred to keep the creation and execution concerns separate.

## Next Steps

* Check out the [`AsyncClient`](https://googleapis.github.io/python-genai/genai.html#genai.client.AsyncClient) class in the Python SDK reference.
* Read more on Python's [`asyncio`](https://docs.python.org/3/library/asyncio.html) library

In [9]:
from google.genai import types

with open('firefighter.jpg', 'rb') as f:
    image_bytes = f.read()

response = client.models.generate_content(
  model='gemini-2.5-flash',
  contents=[
    types.Part.from_bytes(
      data=image_bytes,
      mime_type='image/jpeg',
    ),
    'Caption this image.'
  ]
)

print(response.text)

Here are a few caption options for the image:

**Descriptive:**
*   A simple black and white stick figure drawing of a person looking up at a cat perched on a tree branch.
*   A stick figure wearing a cap stands on the ground, hands on hips, gazing up at a contented cat in a tree.
*   A drawing featuring a large tree with a small, smiling cat on one of its branches, observed by a stick figure person below.

**Narrative/Interpretive:**
*   The classic dilemma: a cat up a tree, and a human trying to figure out how to get it down.
*   "How did you get up there, Mittens?"
*   A moment of contemplation as the stick figure considers how to retrieve the cat from the tree.
*   The cat is enjoying the view, while the person on the ground seems a bit perplexed.

**Humorous:**
*   "I'm not stuck, human. This is my new high-rise apartment." - The cat, probably.
*   Looks like someone forgot their cat-retrieval pole.
*   The ultimate game of 'fetch' where only one player is participating.
*   "Just

In [13]:
from google.genai import types

img_filenames = ["firefighter.jpg", "elephants.jpeg", "jetpack.jpg"]
images_bytes = []

for img_filename in img_filenames:
  with open(img_filename, 'rb') as f:
      images_bytes.append(f.read())



for image_bytes in images_bytes:
  response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=[
      types.Part.from_bytes(
        data=image_bytes,
        mime_type='image/jpeg',
      ),
      'Caption this image.'
    ]
  )

  print(response.text)
  print("---")

Here are a few options for a caption:

**Descriptive:**
*   A simple, childlike drawing depicts a stick figure person standing below a tall tree, looking up at a cat perched on one of its branches.
*   A stick figure person with a hat looks up at a cat sitting on a tree branch.

**Humorous/Relatable:**
*   The classic cat-in-a-tree scenario, stick-figure style.
*   Someone's planning a rescue, or just wondering how the cat got up there.
*   Cat: "I see no problem here." Person: "I beg to differ."
*   The eternal question: how to get the cat down?
*   Looks like a job for the local fire department... or a very patient human.

**Concise:**
*   Cat in a tree.
*   Up a tree.
*   Standoff.
*   Feline high, human low.
---
Here are several caption options for the image, ranging from descriptive to more evocative:

**Concise & Descriptive:**
*   An elephant family, including a young calf, at the edge of a lush green forest.
*   A small herd of Asian elephants near a dirt path in the wilderness

In [14]:
from google.genai import types

async def describe_local_images(images_bytes):
  for image_bytes in images_bytes:
    response = await client.aio.models.generate_content(
      model='gemini-2.5-flash',
      contents=[
        types.Part.from_bytes(
          data=image_bytes,
          mime_type='image/jpeg',
        ),
        'Caption this image.'
      ]
    )

    print(response.text)
    print("---")

img_filenames = ["firefighter.jpg", "elephants.jpeg", "jetpack.jpg"]
images_bytes = []

for img_filename in img_filenames:
  with open(img_filename, 'rb') as f:
      images_bytes.append(f.read())

await describe_local_images(images_bytes)

Here are a few options for captioning this image, ranging from descriptive to more narrative or humorous:

**Descriptive:**
1.  A stick figure person, wearing a hat, stands on the ground looking up at a cat perched on a tree branch.
2.  A simple drawing depicting a cat in a tree and a person on the ground.
3.  A black and white line drawing of a person with hands on hips, gazing up at a cat resting in a tree.

**Narrative/Interpretive:**
4.  The classic dilemma: a cat up a tree, with a person wondering how to get it down.
5.  A stick figure contemplates the challenge of a cat who has climbed too high.
6.  "Now what?" thinks the person below, as the cat relaxes in the tree.

**Humorous:**
7.  Cat: "I'm not coming down." Person: "But... dinner?"
8.  Looks like someone needs a very long stick, or a ladder.
9.  Tree-climbing cat asserts dominance over ground-bound human.
---
Here are a few options for a caption, playing on different aspects of the image:

**Option 1 (Descriptive & Serene):

In [30]:
import asyncio
import time

async def brewCoffee(color):
    print(f"Start brewCoffee({color})")
    await asyncio.sleep(3)
    print(f"End brewCoffee({color})")
    return f"{color} Coffee ready"

async def toastBagel():
    print("Start toastBagel()")
    await asyncio.sleep(2)
    print("End toastBagel()")
    return "Bagel toasted"

async def main():
    start = time.time()
    #result_coffee = brewCoffee()
    #result_bagel = toastBagel()

    #batch = asyncio.gather(brewCoffee("black"), brewCoffee("white"), toastBagel())
    #result_coffee_black, result_coffee_white, result_bagel = await batch

    results = []
    batch = asyncio.gather(brewCoffee("black"), brewCoffee("white"), toastBagel())
    results = await batch


    end_time = time.time()
    elapsed_time = end_time - start
    for result in results:
        print(result)
    #print(result_coffee_black)
    #print(result_coffee_white)
    #print(result_bagel)
    print(elapsed_time)

if __name__ == "__main__":
    await main()


Start brewCoffee(black)
Start brewCoffee(white)
Start toastBagel()
End toastBagel()
End brewCoffee(white)
End brewCoffee(black)
black Coffee ready
white Coffee ready
Bagel toasted
3.026031017303467
